#### Кейс №1 - Приложение знакомств

#### Вы работаете аналитиком в приложении для онлайн-знакомств. Механика приложения следующая: пользователи видят в приложении анкеты друг друга и могут ставить друг другу лайки или дизлайки. Если пользователи поставили друг другу лайк – это называется мэтч, и у пользователей появляется возможность познакомиться.

#### Команда приложения разработала новый алгоритм для поиска наиболее подходящих анкет. Для проверки работы алгоритма был проведен АБ-тест. Все пользователи были разделены на две группы. Пользователи в группе с номером 0 пользовались приложением со старым алгоритмом. Все пользователи в группе 1 пользовались приложением с новым алгоритмом для поиска анкет.

#### Ваша задача – оценить, правда ли, что новый алгоритм улучшил качество сервиса. Для этого нужно выбрать одну или несколько метрик, которые отвечают за качество сервиса, и статистически сравнить эти метрики в двух группах.

#### В данных находится выгрузка логов взаимодействия пользователей друг с другом. Для каждой пары пользователей указано, из какой они группы АБ-теста и случился ли у них мэтч.

#### Результат вашей работы – аналитическое заключение с ответом на вопрос, стоит ли включать новую систему поиска анкет на всех пользователей. #### 

In [17]:
import pandas as pd
import ssl
import urllib.request
import numpy as np
from scipy.stats import chi2_contingency
import scipy.stats as stats

In [3]:
url = "https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/txn7q-CE2zSaPg"
context = ssl._create_unverified_context()
response = urllib.request.urlopen(url, context=context)

dating_data = pd.read_csv(response)
dating_data

,user_id_1,user_id_2,group,is_match
0,79,91,1,1
1,716,353,1,1
2,423,677,0,0
3,658,165,1,1
4,969,155,0,1
...,...,...,...,...
14509,476,631,0,0
14510,455,770,1,0
14511,260,6,1,1
14512,885,812,1,1


group:
    
    0 - контрольная
    1 - тестовая
    
is_match:

    0 - дизлайк( не сошлись)
    1 - лайк (мэтч)

In [4]:
dating_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14514 entries, 0 to 14513
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   user_id_1  14514 non-null  int64
 1   user_id_2  14514 non-null  int64
 2   group      14514 non-null  int64
 3   is_match   14514 non-null  int64
dtypes: int64(4)
memory usage: 453.7 KB


In [9]:
control_all = dating_data.query('group == 0').count()
control_all

user_id_1    4792
user_id_2    4792
group        4792
is_match     4792
dtype: int64

In [10]:
testing_all = dating_data.query('group == 1').count()
testing_all

user_id_1    9722
user_id_2    9722
group        9722
is_match     9722
dtype: int64

In [11]:
control_match = dating_data.query('group == 0').is_match.sum()
control_match

934

In [12]:
testing_match = dating_data.query('group == 1').is_match.sum()
testing_match

3909

У нас категориальные данные, поэтому для теста будем использовать в дальнейшем хи-квадрат, можно еще проверить на критерии Фишера.

В качестве основной метрики используем коэффициент конверсии, т.е. отношение кол-ва мэтчей к общему количеству человек.

In [13]:
control_all - control_match

user_id_1    3858
user_id_2    3858
group        3858
is_match     3858
dtype: int64

In [14]:
testing_all - testing_match

user_id_1    5813
user_id_2    5813
group        5813
is_match     5813
dtype: int64

Хи-квадрат

In [16]:
obs = np.array([[3909, 5813], [934, 3858]])
res = chi2_contingency(obs)
res

Chi2ContingencyResult(statistic=618.6889141576198, pvalue=1.4418299163662586e-136, dof=1, expected_freq=array([[3244.01584677, 6477.98415323],
       [1598.98415323, 3193.01584677]]))

Критерий Фишера

In [18]:
stats.fisher_exact(obs)

SignificanceResult(statistic=2.777670295958516, pvalue=4.1568056557373717e-144)

Оба критерия дают нам понять, что при таком p-value у нас появляются основания отклонить Нулевую гипотезу( о том, что статзначимых различий нет).

Теперь посмотрим на наши метрики Conversion Rate.

In [25]:
control_conversion = (control_match / control_all).mul(100).round(2)
control_conversion

user_id_1    19.49
user_id_2    19.49
group        19.49
is_match     19.49
dtype: float64

In [24]:
testing_conversion = (testing_match / testing_all).mul(100).round(2)
testing_conversion

user_id_1    40.21
user_id_2    40.21
group        40.21
is_match     40.21
dtype: float64

In [26]:
testing_conversion - control_conversion

user_id_1    20.72
user_id_2    20.72
group        20.72
is_match     20.72
dtype: float64

#### Судя по показателям нашей метрики мы смело можем заявить, что новый алгоритм улучшил качество сервиса  на 20.72%. Новую систему поиска анкет на всех пользователей включать стоит!